# **Neo4j 데이터베이스 연결**

In [ ]:
!pip install neo4j

from google.colab import drive
import os

# 구글 드라이브 아누트
drive.mount('/content/drive')

# 디렉터리 변경
os.chdir('워킹 디렉토리 지정')

GRAPHRAG_FOLDER = '아웃풋 디렉토리 지정'

In [ ]:
from neo4j import GraphDatabase
import pandas as pd
import time

# 실제 인스턴스 정보를 입력한ㄷ
NEO4J_URI = "neo4j+s://..."
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "password"
NEO4J_DATABASE = "neo4j"

# **배치 임포트 함수와 제약 조건**

In [ ]:
def batched_import(statement, df, batch_size=1000):
  """
  Import a dataframe into Neo4j using a batched approach.

  Parameters: statement is the Cypher query to execute, df is the dataframe to
  import, and batch_size is the number of rows to import in each batch.
  """

  total = len(df)
  start_s = time.time()

  for start in range(0, total, batch_size):
    batch = df.iloc[start : min(start + batch_size, total)]
    result = driver.execute_query(
        "UNWIND $rows AS value " + statement,
        rows=batch.to_dict("records"),
        database_=NEO4J_DATABASE,
    )

    print(result.summary.counters)

  print(f"{total} rows in {time.time() - start_s} s.")
  return total

# 제약 조건 설정
statements = [
    "\ncreate constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique",
    "\ncreate constraint document_id if not exists for (d:__Document__) require d.id is unique",
    "\ncreate constraint community_id if not exists for (c:__Community__) require c.community is unique",
    "\ncreate constraint entity_id if not exists for (e:__Entity__) require e.id is unique",
    "\ncreate constraint entity_title if not exists for (e:__Entity__) require e.name is unique",
    "\ncreate constraint covariate_title if not exists for (e:__Covariate__) require e.title is unique",
    "\ncreate constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique",
    "\n",
]

for statement in statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)